In [ ]:
!pip install -q --upgrade datasets transformers accelerate evaluate peft # Upgrade to latest
!pip install -q huggingface_hub  # For saving to HF Hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


In [ ]:
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"  # Ignore deprecation; disables WandB
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # TEMP: Sync CUDA for exact error if persists (remove after fix)
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate
import torch
from huggingface_hub import notebook_login
from peft import LoraConfig, get_peft_model, TaskType
notebook_login()  # For HF push

In [ ]:
# Load dataset
dataset = load_dataset("cardiffnlp/x_sensitive")

# Define label keys
label_keys = ['conflictual', 'drugs', 'profanity', 'selfharm', 'sex', 'spam']

# FIXED: Collapse to binary INT label (0/1 for CE loss with num_labels=2)
def collapse_to_binary(example):
    binary_label = 1 if any(example[key] == 1 for key in label_keys) else 0
    example['labels'] = int(binary_label)  # INT: For CrossEntropyLoss (long tensors)
    return example

# Apply to all splits
dataset = dataset.map(collapse_to_binary)

# Quick stats
print("Binary label distribution:")
for split in dataset:
    labels = dataset[split]['labels']
    print(f"{split}: Sensitive {np.sum(labels)} / {len(labels)} ({np.mean(labels)*100:.1f}%)")

# Tokenizer
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=False, max_length=128)

# Tokenize splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text", "keyword"] + label_keys + [k for k in dataset['train'].features if '_highlight' in k] + ['#labels'])
tokenized_datasets.set_format("torch")

# Verify: Print a sample (labels should be int/long)
print("\nSample after tokenization:")
sample = tokenized_datasets["train"][0]
print(f"Keys: {list(sample.keys())}")
print(f"Labels type/value: {type(sample['labels'])} / {sample['labels']}")

# FIXED: Load model for binary (2 classes: 0=not, 1=sensitive; CE loss)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # 2 classes for CrossEntropyLoss
    problem_type="single_label_classification"
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Metrics (argmax for 2-class preds)
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # FIXED: Argmax for CE (no sigmoid)
    return {
        'accuracy': accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        'f1': f1.compute(predictions=predictions, references=labels, average='binary')['f1']
    }

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Binary label distribution:
train: Sensitive 2381 / 5000 (47.6%)
validation: Sensitive 436 / 1000 (43.6%)
test: Sensitive 1082 / 2000 (54.1%)


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


Sample after tokenization:
Keys: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
Labels type/value: <class 'torch.Tensor'> / 0


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_full = TrainingArguments(
    output_dir="./x-sensitive-deberta-binary",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None,  # No logging
    push_to_hub=True,
    hub_model_id="faketut/x-sensitive-deberta-binary",
)

trainer_full = Trainer(
    model=model,
    args=training_args_full,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,  # No deprecation warning
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start full fine-tuning
trainer_full.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluate full model on test set
test_results_full = trainer_full.evaluate(tokenized_datasets["test"])
print("Full Fine-Tuning Test Results:", test_results_full)

# Save final checkpoint locally and to HF Hub
trainer_full.save_model("./x-sensitive-deberta-binary-final")

# Test inference with full model
from transformers import pipeline
classifier_full = pipeline("text-classification", model="./x-sensitive-deberta-full-final", return_all_scores=True)
print("Full Model Inference:", classifier_full("This is so fucking cool!"))

Test Results: {'eval_loss': 0.4083416759967804, 'eval_accuracy': 0.8265, 'eval_f1': 0.8288110508140109, 'eval_runtime': 14.5812, 'eval_samples_per_second': 137.163, 'eval_steps_per_second': 8.573, 'epoch': 3.0}


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-binary/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...798548.4746f84f0bd9.960.4: 100%|##########| 5.39kB / 5.39kB            

...

  ...798300.4746f84f0bd9.960.2: 100%|##########| 5.40kB / 5.40kB            

  ...9396.4746f84f0bd9.12355.0: 100%|##########| 6.98kB / 6.98kB            

  ...798654.4746f84f0bd9.960.5: 100%|##########| 5.39kB / 5.39kB            

  ...-deberta-binary/spm.model: 100%|##########| 2.46MB / 2.46MB            

  ...-binary/model.safetensors:   3%|3         | 25.1MB /  738MB            

In [ ]:
# Reload base model for LoRA (fair comparison from scratch)
full_model_repo = "faketut/x-sensitive-deberta-binary"
model_lora_base = AutoModelForSequenceClassification.from_pretrained(
    full_model_repo,
    local_files_only=False,
)

# LoRA Config (target modules for DeBERTa-v3 attention layers)
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["query_proj", "value_proj"],  # Key DeBERTa attention modules
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Apply LoRA
model_lora = get_peft_model(model_lora_base, lora_config)
model_lora.print_trainable_parameters()  # Show trainable params %

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

trainable params: 591,362 || all params: 185,015,044 || trainable%: 0.3196


In [ ]:
# LoRA Training Args (same as full for fair comparison)
training_args_lora = TrainingArguments(
    output_dir="./x-sensitive-deberta-lora",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None,  # No logging
    push_to_hub=True,
    hub_model_id="faketut/x-sensitive-deberta-lora",
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start LoRA fine-tuning
trainer_lora.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.324636,0.882000,0.861827


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.324636,0.882000,0.861827
2,0.253100,0.331084,0.878000,0.860092
3,0.253100,0.329818,0.880000,0.861432


TrainOutput(global_step=939, training_loss=0.2589283659816169, metrics={'train_runtime': 277.4502, 'train_samples_per_second': 54.064, 'train_steps_per_second': 3.384, 'total_flos': 605838392202432.0, 'train_loss': 0.2589283659816169, 'epoch': 3.0})

In [ ]:
# Evaluate LoRA model on test set
test_results_lora = trainer_lora.evaluate(tokenized_datasets["test"])
print("LoRA Test Results:", test_results_lora)

# Save LoRA model (adapters)
trainer_lora.save_model("./x-sensitive-deberta-lora-final")

# Test inference with LoRA model (merge for pipeline)
from transformers import pipeline
model_lora_merged = model_lora.merge_and_unload()  # Merge adapters
classifier_lora = pipeline("text-classification", model=model_lora_merged, tokenizer=tokenizer, return_all_scores=True)
print("LoRA Model Inference:", classifier_lora("This is so fucking cool!"))

Device set to use cuda:0


LoRA Model Inference: [[{'label': 'LABEL_0', 'score': 0.0076930588111281395}, {'label': 'LABEL_1', 'score': 0.9923070073127747}]]


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# =============================================================================
# Comparison Summary
# =============================================================================
print("\n=== Comparison Summary ===")
print("Full Fine-Tuning Test - Accuracy:", test_results['eval_accuracy'], "F1:", test_results['eval_f1'])
print("LoRA (on Full) Test - Accuracy:", test_results_lora['eval_accuracy'], "F1:", test_results_lora['eval_f1'])


=== Comparison Summary ===


NameError: name 'test_results' is not defined